In [2]:
import pandas as pd
import json
import boto3
import awswrangler as wr
from tool import find_unique_combinations
import sqlite3
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

boto3.setup_default_session(profile_name='vtn')

In [25]:
# db_path = "/Users/pongv/Desktop/tradesquareltd/vtn/TesterScript/vtnlake_onedrive_new.db"
db_path = "vtnlake_onedrive_new.db"
table_name = "gl_line_full"
conn = sqlite3.connect(db_path)
head20230801 = pd.read_sql_query(f"SELECT * FROM {table_name} limit 1_000_000", conn)

head20230801

,business_unit,invoice_no_receipt_no,invoice_receipt_line_no,billing_sub_group,item_code,item_description,package_code,revenue_account,discount_line_account,department_code,first_location,patient_location,receiving_location,revenue_subtype,amount,discount_amount,doctor_group,doctor_type,doctor_code,doctor_name,quantity,date,episode_number,nationality,hn,transaction_no,order_item_id,order_item_sub_id,episode_id,patient_id,bill_row_id,file_name
0,VTN01,CI67000972,35,11010002,MLVSTER02,** Sterile water for inj 1000,None,400002,480002.0,110010,2MED,2ICU1,1PHI,IPD,1210.0,847.0,NaN,NUFUL01,9006343,มาริสา ภู่สุวรรณ พว.,1000.0,15/01/2024,I24-000413,TH,97-074923,None,13888650,3,8515607,74923,51435895,REVLVTN0120240116_20240124T155744.txt
1,VTN01,CI67000972,273,11010002,MLVPIGN01,NSS inj (50 ml)[ND]_UC,None,400002,480002.0,110010,2MED,1RIS,2PHO6,IPD,170.0,119.0,92635.0,DRMS01,92635,ชานนท์ พิริยะวัฒนกุล นพ.,50.0,15/01/2024,I24-000413,TH,97-074923,None,13888307,27,8515607,74923,51435895,REVLVTN0120240116_20240124T155744.txt
2,VTN01,CI67000972,781,11010002,MLVPIGN02,NSS Piggy Back inj 100ml [ND],RAD06,400002,480002.0,110010,2MED,1RIS,1PHO1,IPD,120.0,84.0,NaN,NUFUL01,9009821,กฤษณา อุทากิจ คุณ,100.0,15/01/2024,I24-000413,TH,97-074923,None,13888650,78,8515607,74923,51435895,REVLVTN0120240116_20240124T155744.txt
3,VTN01,CI67000972,151,11120000,NUOPDNURC029,ค่าการพยาบาล ใช้เวลาไม่เกิน 10,VCC0000002,455001,485501.0,110010,2MED,2ER,CASO1,IPD,100.0,10.0,NaN,NUFUL01,9009809,ฝนแก้ว รัตนดิลก ณ ภูเก็ต พว.,1.0,15/01/2024,I24-000413,TH,97-074923,None,13888307,15,8515607,74923,51435895,REVLVTN0120240116_20240124T155744.txt
4,VTN01,CI67000972,28,11120000,OPDNE,ค่าบริการพยาบาลผู้ป่วยอุบัติเห,None,455001,485501.0,110010,2MED,2MED,2ER,IPD,500.0,50.0,NaN,None,None,None,1.0,15/01/2024,I24-000413,TH,97-074923,None,13888307,2,8515607,74923,51435895,REVLVTN0120240116_20240124T155744.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,VTN01,D67001366,309,11140001,DRUG4CHARGE,ค่ายาชุดเหมาจ่าย,GI00000014,400002,480002.0,110070,2GI,2GI,CASO1,OPD,1100.0,0.0,92435.0,DRMS01,92435,ศศิพิมพ์ จามิกร พญ.,1.0,28/01/2024,O24-018749,YM,24-001902,None,14012618,30,8530942,1472728,51470401,REVLVTN0120240128_20240131T155625.txt
999996,VTN01,D67001366,41,11140001,MOXXYLO02_M01,Xylocaine (R)Jelly (คิดเงินต่,GI00000014,400001,480001.0,110070,2GI,2GI,CASO1,OPD,225.0,0.0,92435.0,DRMS01,92435,ศศิพิมพ์ จามิกร พญ.,25.0,28/01/2024,O24-018749,YM,24-001902,None,14012618,4,8530942,1472728,51470401,REVLVTN0120240128_20240131T155625.txt
999997,VTN01,D67001366,421,11010001,MOTVOCI01,Vocinti tab 10 mg(R)[ND],None,400001,480001.0,110070,2GI,2GI,1PHO1,OPD,1024.0,0.0,92435.0,DRMS01,92435,ศศิพิมพ์ จามิกร พญ.,8.0,28/01/2024,O24-018749,YM,24-001902,None,14012618,42,8530942,1472728,51470401,REVLVTN0120240128_20240131T155625.txt
999998,VTN01,D67001366,432,11010001,MOTMOTI01,Motilium-M tab 10 mg (R) [C]_U,None,400001,480001.0,110070,2GI,2GI,1PHO1,OPD,192.0,0.0,92435.0,DRMS01,92435,ศศิพิมพ์ จามิกร พญ.,8.0,28/01/2024,O24-018749,YM,24-001902,None,14012618,43,8530942,1472728,51470401,REVLVTN0120240128_20240131T155625.txt


In [12]:
head20230801.columns = [
                    'business_unit',
                    'invoice_no_receipt_no',
                    'invoice_receipt_line_no',
                    'billing_sub_group',
                    'item_code',
                    'item_description',

                    'package_code',
                    'revenue_account',
                    'discount_line_account',
                    'department_code',
                    'first_location',
                    'patient_location',

                    'receiving_location',
                    'revenue_subtype',
                    'amount',
                    'discount_amount',
                    'doctor_group',
                    'doctor_type',
                    'doctor_code',
                    'doctor_name',

                    'quantity',
                    'date',
                    'episode_number',
                    'nationality',
                    'hn',

                    'transaction_no',
                    'order_item_id',
                    'order_item_sub_id',
                    'bill_row_id',
                    'episode_id',
                    'patient_id',
                    'file_name'
]

In [13]:
head20230801

,business_unit,invoice_no_receipt_no,invoice_receipt_line_no,billing_sub_group,item_code,item_description,package_code,revenue_account,discount_line_account,department_code,first_location,patient_location,receiving_location,revenue_subtype,amount,discount_amount,doctor_group,doctor_type,doctor_code,doctor_name,quantity,date,episode_number,nationality,hn,transaction_no,order_item_id,order_item_sub_id,bill_row_id,episode_id,patient_id,file_name
0,VTN01,CI67000972,35,11010002,MLVSTER02,** Sterile water for inj 1000,None,400002,480002.0,110010,2MED,2ICU1,1PHI,IPD,1210.0,847.0,NaN,NUFUL01,9006343,มาริสา ภู่สุวรรณ พว.,1000.0,15/01/2024,I24-000413,TH,97-074923,None,13888650,3,8515607,74923,51435895,REVLVTN0120240116_20240124T155744.txt
1,VTN01,CI67000972,273,11010002,MLVPIGN01,NSS inj (50 ml)[ND]_UC,None,400002,480002.0,110010,2MED,1RIS,2PHO6,IPD,170.0,119.0,92635.0,DRMS01,92635,ชานนท์ พิริยะวัฒนกุล นพ.,50.0,15/01/2024,I24-000413,TH,97-074923,None,13888307,27,8515607,74923,51435895,REVLVTN0120240116_20240124T155744.txt
2,VTN01,CI67000972,781,11010002,MLVPIGN02,NSS Piggy Back inj 100ml [ND],RAD06,400002,480002.0,110010,2MED,1RIS,1PHO1,IPD,120.0,84.0,NaN,NUFUL01,9009821,กฤษณา อุทากิจ คุณ,100.0,15/01/2024,I24-000413,TH,97-074923,None,13888650,78,8515607,74923,51435895,REVLVTN0120240116_20240124T155744.txt
3,VTN01,CI67000972,151,11120000,NUOPDNURC029,ค่าการพยาบาล ใช้เวลาไม่เกิน 10,VCC0000002,455001,485501.0,110010,2MED,2ER,CASO1,IPD,100.0,10.0,NaN,NUFUL01,9009809,ฝนแก้ว รัตนดิลก ณ ภูเก็ต พว.,1.0,15/01/2024,I24-000413,TH,97-074923,None,13888307,15,8515607,74923,51435895,REVLVTN0120240116_20240124T155744.txt
4,VTN01,CI67000972,28,11120000,OPDNE,ค่าบริการพยาบาลผู้ป่วยอุบัติเห,None,455001,485501.0,110010,2MED,2MED,2ER,IPD,500.0,50.0,NaN,None,None,None,1.0,15/01/2024,I24-000413,TH,97-074923,None,13888307,2,8515607,74923,51435895,REVLVTN0120240116_20240124T155744.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,VTN01,CI66008311,1629,11020001,O4229SILDRA10,2014-0102 SILICONE FLAT DRAIN,None,410001,481001.0,110040,2WHC,1OR,1OR,IPD,936.0,0.0,NaN,None,None,None,1.0,21/05/2023,I23-003635,TH,23-009043,None,11711172,162,8232583,1442122,50826689,REVLVTN01.20230528
99996,VTN01,CI66008311,1631,11020001,O4227SILRE200,Silicone CWV Reservoir 200cc.,None,410001,481001.0,110040,2WHC,1OR,1OR,IPD,1338.0,0.0,NaN,None,None,None,1.0,21/05/2023,I23-003635,TH,23-009043,None,11711172,163,8232583,1442122,50826689,REVLVTN01.20230528
99997,VTN01,CI66008311,1643,11020001,S4229VIC19468,VICRYL 1-0 [VCP9468H],None,410001,481001.0,110040,2WHC,1OR,1MMDS,IPD,732.0,0.0,NaN,None,None,None,1.0,21/05/2023,I23-003635,TH,23-009043,None,11711172,164,8232583,1442122,50826689,REVLVTN01.20230528
99998,VTN01,CI66008311,1655,11020001,S4229VIC39114,VICRYL 3-0 [W9114],None,410001,481001.0,110040,2WHC,1OR,1MMDS,IPD,2196.0,0.0,NaN,None,None,None,3.0,21/05/2023,I23-003635,TH,23-009043,None,11711172,165,8232583,1442122,50826689,REVLVTN01.20230528


In [14]:
head20230801.apply(lambda x: len(x.unique()))

business_unit                 1
invoice_no_receipt_no      7820
invoice_receipt_line_no    6181
billing_sub_group           112
item_code                  3825
item_description           3785
package_code                922
revenue_account              65
discount_line_account        65
department_code              40
first_location               54
patient_location             72
receiving_location           79
revenue_subtype               2
amount                     3611
discount_amount            3820
doctor_group                387
doctor_type                  33
doctor_code                 743
doctor_name                 752
quantity                    195
date                        262
episode_number             6844
nationality                  72
hn                         5368
transaction_no                1
order_item_id              7060
order_item_sub_id          3782
bill_row_id                6844
episode_id                 5368
patient_id                 7711
file_nam

In [30]:
head20230801[['invoice_no_receipt_no','invoice_receipt_line_no','order_item_sub_id','order_item_id']].value_counts() #'Document type'???

invoice_no_receipt_no  invoice_receipt_line_no  order_item_sub_id  order_item_id
CI66000401             25786                    2578               10203423         1
O66008066              30                       3                  10538839         1
                       50                       5                  10538839         1
                       60                       6                  10538839         1
                       70                       7                  10538839         1
                                                                                   ..
CO66180634             32                       3                  12848149         1
                       42                       4                  12848149         1
                       52                       5                  12848149         1
                       62                       6                  12848149         1
O67161740              66                       6          

In [18]:
head20230801[head20230801['invoice_no_receipt_no']=='I62000279']

,business_unit,invoice_no_receipt_no,invoice_receipt_line_no,billing_sub_group,item_code,item_description,package_code,revenue_account,discount_line_account,department_code,first_location,patient_location,receiving_location,revenue_subtype,amount,discount_amount,doctor_group,doctor_type,doctor_code,doctor_name,quantity,date,episode_number,nationality,hn,transaction_no,order_item_id,order_item_sub_id,bill_row_id,episode_id,patient_id,file_name


In [28]:
find_unique_combinations(head20230801, [
    # 'invoice_no_receipt_no','invoice_receipt_line_no','order_item_sub_id','order_item_id'
    'invoice_no_receipt_no',
    'invoice_receipt_line_no',
    'item_code',
])

Unique combination: ('invoice_no_receipt_no', 'invoice_receipt_line_no', 'item_code')
End of search


In [29]:
find_unique_combinations(head20230801, [
    'invoice_no_receipt_no','invoice_receipt_line_no','order_item_sub_id','order_item_id'
])


Unique combination: ('invoice_no_receipt_no', 'order_item_sub_id', 'order_item_id')
Unique combination: ('invoice_no_receipt_no', 'invoice_receipt_line_no', 'order_item_sub_id', 'order_item_id')
End of search
